In [21]:
pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
from bs4 import BeautifulSoup
import requests

In [23]:
url = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [24]:
table = soup.find_all("table")[2]

In [25]:
table.find_all("th")

[<th>Rank
 </th>,
 <th>Name
 </th>,
 <th>Industry
 </th>,
 <th>Profits<br/>(USD millions)
 </th>]

In [26]:
col = [tab.text.strip() for tab in table.find_all("th")]
col

['Rank', 'Name', 'Industry', 'Profits(USD millions)']

In [27]:
import pandas as pd

In [28]:
df = pd.DataFrame(columns = col)

In [29]:
data = table.find_all("tr")
for dat in data[1:]:
    row_data = (dat.find_all("td"))
    ind_row_data = [row_dat.text.strip() for row_dat in row_data]
    print(ind_row_data)
    lent = len(df)
    df.loc[lent] = ind_row_data

['1', 'Apple', 'Electronics', '96,995']
['2', 'Berkshire Hathaway', 'Conglomerate', '96,223']
['3', 'Alphabet', 'Technology', '73,795']
['4', 'Microsoft', 'Technology', '72,361']
['5', 'JPMorgan Chase', 'Financials', '49,552']
['6', 'Meta Platforms', 'Technology', '39,098']
['7', 'ExxonMobil', 'Petroleum industry', '36,010']
['8', 'Johnson & Johnson', 'Pharmaceutical industry', '35,153']
['9', 'Amazon', 'Retail/Technology', '30,425']
['10', 'Nvidia', 'Technology', '29,760']


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Rank                   10 non-null     object
 1   Name                   10 non-null     object
 2   Industry               10 non-null     object
 3   Profits(USD millions)  10 non-null     object
dtypes: object(4)
memory usage: 400.0+ bytes


In [31]:
# df["Rank"] = df["Rank"].astype(int)
df["Profits(USD millions)"] = df["Profits(USD millions)"].str.replace(",","")
df["Profits(USD millions)"] = df["Profits(USD millions)"].astype(int)

In [32]:
df

,Rank,Name,Industry,Profits(USD millions)
0,1,Apple,Electronics,96995
1,2,Berkshire Hathaway,Conglomerate,96223
2,3,Alphabet,Technology,73795
3,4,Microsoft,Technology,72361
4,5,JPMorgan Chase,Financials,49552
5,6,Meta Platforms,Technology,39098
6,7,ExxonMobil,Petroleum industry,36010
7,8,Johnson & Johnson,Pharmaceutical industry,35153
8,9,Amazon,Retail/Technology,30425
9,10,Nvidia,Technology,29760


In [33]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------ --------------------- 0.5/1.2 MB 929.6 kB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [34]:
import psycopg2

In [52]:
host = "localhost"
port = "5432"
user = "postgres"
password = "YbabyLO@02#"

conn = psycopg2.connect(
    user = user,
    password = password,
    host = host,
    port = port
)

conn.autocommit = True
cur = conn.cursor()


In [53]:
create_db = '''"create database companies"'''
cur.execute(create_db)

In [54]:
cur.close()
conn.close()

In [57]:
host = "localhost"
port = "5432"
user = "postgres"
password = "YbabyLO@02#"
db = "companies"

conn = psycopg2.connect(
    dbname = db,
    user = user,
    password = password,
    host = host,
    port = port
)

conn.autocommit = True
cur = conn.cursor()


In [62]:
create_table = '''create table rich_by_price(
    "Rank" int primary key,
    "Name" varchar(50),
    "Industry" varchar(50),
    "Profits" int
    );
'''

cur.execute(create_table)

cur.close()

In [71]:
conn.commit()

In [68]:
cur = conn.cursor()
insert_into = '''insert into rich_by_price( "Rank", "Name", "Industry", "Profits")
Values (%s, %s, %s, %s)
'''

for index,row in df.iterrows():
    cur.execute(insert_into,list(row))


cur.close()

In [73]:
cur = conn.cursor()
select_from = ''' select * from rich_by_price'''

cur.execute(select_from)

rowss = cur.fetchall()

for row in rowss:
    print(row) 

(1, 'Apple', 'Electronics', 96995)
(2, 'Berkshire Hathaway', 'Conglomerate', 96223)
(3, 'Alphabet', 'Technology', 73795)
(4, 'Microsoft', 'Technology', 72361)
(5, 'JPMorgan Chase', 'Financials', 49552)
(6, 'Meta Platforms', 'Technology', 39098)
(7, 'ExxonMobil', 'Petroleum industry', 36010)
(8, 'Johnson & Johnson', 'Pharmaceutical industry', 35153)
(9, 'Amazon', 'Retail/Technology', 30425)
(10, 'Nvidia', 'Technology', 29760)
